Empezamos a trabajar con el dataset proporcionado sin demora ninguna, recordando que Larry nos había mostrado particular interés en el distrito de Abbotsford, ya que vivía la mejor amiga de su mmujer, pero bueno empezamos primero viendo nuestro dataset

In [54]:
import numpy as np 
from numpy import sqrt, abs, round
import pandas as pd
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
import math
from scipy.stats import norm, uniform, zscore
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_big = pd.read_csv("housing_Melbourne_big.csv")
df=pd.read_csv("housing_Melbourne.csv")
df_big

In [ ]:
df_big.info()

In [ ]:
df_big.describe()

In [ ]:
num_col = df_big.select_dtypes(include=np.number).columns
print("Numerical columns: \n",num_col)

cat_col = df_big.select_dtypes(exclude=np.number).columns
print("Categorical columns: \n",cat_col)

In [ ]:
df_big.drop(["Address","Date","Postcode","Date"], axis=1,inplace=True)
df_big.dropna(inplace=True)

# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column . 

df_big['Suburb']= label_encoder.fit_transform(df_big['Suburb'])
df_big['Type']= label_encoder.fit_transform(df_big['Type'])
df_big['Method']= label_encoder.fit_transform(df_big['Method'])
df_big['SellerG']= label_encoder.fit_transform(df_big['SellerG'])
df_big['Regionname']= label_encoder.fit_transform(df_big['Regionname'])
df_big['CouncilArea']= label_encoder.fit_transform(df_big['CouncilArea'])
  
df_big.head()


In [ ]:
df_big.hist(figsize=(18,14))

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Price Distribution Plot')
sns.histplot(df_big['Price'])

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_big.corr(),cmap='RdYlGn', annot=True)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="Lattitude", y="Longtitude", data=df_big, hue="Price", palette="coolwarm")

In [ ]:
df.head(10)

### En este estudio, intentaré realizar una prueba básica de hipótesis para el distrito de Abbotsford en Melbourne.

- **Hipótesis nula (H₀):** Los precios de las casas en Abbotsford-Melbourne son más caros que en Melbourne en general.  
  
- **Hipótesis alternativa (H₁):** Los precios de las casas en Abbotsford-Melbourne no son más caros que en Melbourne en general.  

In [ ]:
df_without_null_prices = df[~df["Price"].isnull()]
prices = df_without_null_prices["Price"]
prices

In [ ]:
in_abbotsford = df_without_null_prices.loc[df["Suburb"] == "Abbotsford"]
prices_in_abbotsford = in_abbotsford["Price"]
prices_in_abbotsford

In [ ]:
prices_in_abbotsford_mean = prices_in_abbotsford.mean()
prices_in_abbotsford_mean

In [ ]:
prices_mean = prices.mean()
prices_mean

In [ ]:
prices_in_abbotsford_std = prices_in_abbotsford.std()
prices_in_abbotsford_std

In [ ]:
prices_std = prices.std()
prices_std

In [ ]:
prices_in_abbotsford_len = len(prices_in_abbotsford)
prices_in_abbotsford_len
prices_len = len(prices)
prices_len

In [50]:
min_price = prices.min()
max_price = prices.max()
min_abbotsford_price = prices_in_abbotsford.min()
max_abbotsford_price = prices_in_abbotsford.max()

In [ ]:
standard_error = prices_std / math.sqrt(prices_in_abbotsford_len)
standard_error

### Como se declaró en la hipótesis, se sabe que Abbotsford es un distrito de Melbourne. Primero se calcularon la media y la desviación estándar de la población (Melbourne) y de la muestra (Abbotsford):

- **Media de los precios de las casas en Melbourne:** $997,898$  
  
- **Desviación estándar de los precios de las casas en Melbourne:** $593,498$  
  
- **Media de los precios de las casas en Abbotsford:** $1,040,200$  
  
- **Desviación estándar de los precios de las casas en Abbotsford:** $374,652$  

### Después de esto, se determinó cómo la media de Abbotsford se desvía de la población total (Melbourne) en términos de unidades de error estándar. El resultado del cálculo fue:  
- **Unidades de error estándar (se):** $0.84$  
  
- **Valor p:** $0.39$  

In [58]:
mean_diff = prices_in_abbotsford_mean - prices_mean

In [ ]:
standard_errors_away = mean_diff / standard_error
standard_errors_away

In [ ]:
p_value = 2 * (1-norm.cdf(abs(standard_errors_away)))
p_value

In [61]:
mu = prices_mean
sigma = prices_std / sqrt(prices_in_abbotsford_len)

In [62]:
mu = prices_mean
sigma = prices_std / sqrt(prices_in_abbotsford_len)

In [ ]:

x = np.linspace(mu - 3*sigma, mu + 3*sigma, len(prices))
plt.plot(x, norm.pdf(x, mu, sigma))
plt.axvline(x=prices_in_abbotsford_mean, color="r")
plt.show()

In [ ]:
sigma

In [ ]:
value = np.random.normal(loc=prices_mean,scale=sigma,size=len(prices))
sns.displot(value, kde=True)
plt.axvline(prices_in_abbotsford_mean, color='red') # Plot a vertical line at corresponding x

Entonces, la media de los precios de las casas en Abbotsford fue $0,84$ unidades de error estándar más alta que la media de la población. Preferí alinearme con la convención general de un nivel de error estándar de $1,645$ y un nivel de valor `p` de $0,05$. Dado que nuestros 

valores de error estándar $(0,84 < 1,645)$

 y el valor 
 
 $p (0,39 > 0,05)$ 
 
 no cumplen los criterios, la hipótesis de que los precios de las casas en Abbotsford-Melbourne son más altos que en Melbourne en general es rechazada. Podemos asumir que las casas en Abbotsford no son significativamente más caras que las de la población total, es decir, Melbourne. 

Dado que la unidad de error estándar se encuentra en $49.805$ dólares, y la media del precio de las casas en Abbotsford está a $0,84$ unidades de error estándar de distancia, el cálculo es:  

$49.805$ dólares $*$ $0,84 = 41.836$ dólares. Esto significa que la media de los precios de las casas en Abbotsford está $41.836$ dólares por encima de la media de los precios de las casas en Melbourne.  

Podemos observar los gráficos de resultados más arriba. Estos no representan una `distribución normal estandarizada`, por lo que es necesario explicarlo. En los gráficos trazados, 1.0 significa 1,0 billones de dólares, y 1.1 también significa 1,1 billones de dólares. Desde la línea roja, que representa la media de los precios de las casas en Abbotsford, podemos observar claramente en la gráfica que está desviada $41.836$ dólares.